In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# image

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from abc import ABC, abstractmethod
import numpy as np

In [ ]:
#| export
class Image(ABC):
    "Abstract base class for image rendering and manipulation."

    def __init__(self, width: int = 800, height: int = 800):
        "Initialize an image with a given resolution."
        if width <= 0 or height <= 0:
            raise ValueError("width and height must be positive integers.")
        self._width = width
        self._height = height
        # TODO: use float instead of np.float32
        self._image = np.zeros((self._height, self._width), dtype=float)

    @property
    def resolution(self) -> tuple[int, int]:
        "Get the resolution (width, height) of the image."
        return self._width, self._height

    @resolution.setter
    def resolution(self, res: tuple[int, int]):
        "Set a new resolution for the image and reset the image array."
        w, h = res
        if w <= 0 or h <= 0:
            raise ValueError("Resolution must be positive integers.")
        self._width = w
        self._height = h
        # TODO: refactor not to reset image?
        self._image = np.zeros((self._height, self._width), dtype=float)  # Reset image

    def render(self) -> np.ndarray:
        "Render the image using subclass-defined logic."
        before = self._image.copy()
        self._render()
        if np.array_equal(before, self._image):
            raise RuntimeError("Subclasses must modify self._image during rendering.")
        return self._image

    @abstractmethod
    def _render(self):
        "Abstract method that subclasses must implement to generate image data."
        pass

    def image(self) -> np.ndarray:
        "Return the current image data."
        return self._image

    def plot(self, cmap: str = "turbo"):
        "Display the image using matplotlib."
        plt.figure(figsize=(10, 10))
        plt.imshow(self._image, cmap=cmap)
        plt.axis('off')
        plt.show()

    def equalize_histogram(self):
        "Apply histogram equalization to the image."
        hist, bins = np.histogram(self._image.flatten(), bins=512, density=True)
        cdf = hist.cumsum()  # cumulative distribution function
        cdf = cdf / cdf[-1]  # normalize to [0,1]
        # Use linear interpolation of cdf to find new pixel values
        image_flat = np.interp(self._image.flatten(), bins[:-1], cdf)
        self._image = image_flat.reshape(self._image.shape)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()